In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number, desc, rank, dense_rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName('test').getOrCreate()
data = [['Shiv','Shoes',100.24,'Bata'],['Raju','Bag', 203.45,"Bata"],['Sukesh','Perfume', 102.00, 'Archies'],["Rajendra", 'Books', 119.00, "Archies"],['Rajendra','Pens', 302.00, 'Archies']]
schema = ['customer_name', 'product_name','amount', 'vendor_name']
df  = spark.createDataFrame(data=data, schema=schema)
df.createOrReplaceTempView('sales')

sql normal select query
df = spark.sql("select customer_name, product_name, amount, vendor_name from sales")
df.show()

##########
## Row Number:: row_number function used to generate unique number for each row depend on some specific column field
##########

#example of sql query on row_number built in function 
# df = spark.sql( 'select row_number() over(order by customer_name) as order_number, customer_name, product_name, amount, vendor_name from sales').show()

#Example of pysaprk query on row_number built in function 
# df = df.select(row_number().over(Window.orderBy('customer_name')).alias('order_number'), 'customer_name', 'product_name', 'amount', 'vendor_name').show()


###########
## Partititon By:: Partition by used to partiton into groups depend on perticular column values & provide rank depend on individual group.
###########

#example of sql query on row_number & partition by build in function
# df = spark.sql('select row_number() over(order by vendor_name) as order_number, row_number() over(partition by vendor_name order by vendor_name) as partition_order_number, customer_name, product_name, amount, vendor_name from sales').show()

#example of pyspark query on row_number & partition by build in function
# df = df.select(row_number().over(Window.orderBy('vendor_name')).alias('order_number'), row_number().over(Window.partitionBy('vendor_name').orderBy('vendor_name')).alias('partition_order_number'), 'customer_name', 'product_name', 'amount', 'vendor_name').show()

##############
### Rank:: Used to give the unique rank value for each row bas on perticular column field
##############

#Exaample of sql query on rank function
# df = spark.sql('select row_number() over(order by customer_name) as row_number, row_number() over(partition by customer_name order by customer_name) as partition_row_number,rank() over(order by customer_name) as rank_number, customer_name, product_name, amount, vendor_name from sales').show()

#Example of pyspark sql on rank function
# df = df.select(row_number().over(Window.orderBy('customer_name')).alias('order_number'), row_number().over(Window.partitionBy('vendor_name').orderBy('vendor_name')).alias('partition_order_number'), rank().over(Window.orderBy('customer_name')).alias('rank_number'), 'customer_name', 'product_name', 'amount', 'vendor_name').show()


#################
## DenseRank:: Used to give the unique rank value for each diffenent value with out spikking the rank value sequnence
#################

#Exaample of sql query on rank function
# df = spark.sql('select row_number() over(order by customer_name) as row_number, row_number() over(partition by customer_name order by customer_name) as partition_row_number,rank() over(order by customer_name) as rank_number, dense_rank() over(order by customer_name) as desne_rank_number,customer_name, product_name, amount, vendor_name from sales').show()

#Example of pyspark sql on rank function
df = df.select(row_number().over(Window.orderBy('customer_name')).alias('order_number'), row_number().over(Window.partitionBy('vendor_name').orderBy('vendor_name')).alias('partition_order_number'), rank().over(Window.orderBy('customer_name')).alias('rank_number'),dense_rank().over(Window.orderBy('customer_name')).alias('dense_rank_number'), 'customer_name', 'product_name', 'amount', 'vendor_name').show()



+------------+----------------------+-----------+-----------------+-------------+------------+------+-----------+
|order_number|partition_order_number|rank_number|dense_rank_number|customer_name|product_name|amount|vendor_name|
+------------+----------------------+-----------+-----------------+-------------+------------+------+-----------+
|           1|                     2|          1|                1|     Rajendra|       Books| 119.0|    Archies|
|           2|                     3|          1|                1|     Rajendra|        Pens| 302.0|    Archies|
|           3|                     2|          3|                2|         Raju|         Bag|203.45|       Bata|
|           4|                     1|          4|                3|         Shiv|       Shoes|100.24|       Bata|
|           5|                     1|          5|                4|       Sukesh|     Perfume| 102.0|    Archies|
+------------+----------------------+-----------+-----------------+-------------+-------